In [7]:
import pandas as pd
import chardet
import numpy as np


# nltk.download('stopwords')


class JapanWordFreq:
    def __init__(self):
        pass


with open("D:/05.japanproject/jp/freq/spoken_freq_nlb.txt", "rb") as f:
    result = chardet.detect(f.read())
    print(f"检测到编码: {result['encoding']}")

spoken_freq_nlb_txt_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb.txt"
spoken_freq_nlb_csv_path = "D:/05.japanproject/jp/freq/spoken_freq_nlb.csv"

df = pd.read_csv(
    spoken_freq_nlb_txt_path,
    header=None,
    encoding="utf-8-sig",
    sep="\t",
    on_bad_lines="skip",
    quotechar='"',
    engine="python",
)

df.drop([0, 7], axis=1, inplace=True)

检测到编码: UTF-8-SIG


In [8]:
df.columns = ["headword", "type", "reading", "freq", "freq_per", "rank"]
df

,headword,type,reading,freq,freq_per,rank
0,.,記号,．,"989,922",8.7065,0.99900
1,(,記号,（,"851,151",7.4860,0.99890
2,*,NaN,*,"329,229",2.8956,0.99728
3,*,記号,＊,"306,861",2.6989,0.99710
4,/,記号,／,"199,654",1.7560,0.99524
...,...,...,...,...,...,...
98172,｝,記号,｝,"3,281",0.0289,0.86545
98173,～,記号,～,"152,926",1.3450,0.99366
98174,￡,記号,ポンド,82,0.0007,0.46012
98175,￣,記号,￣,804,0.0071,0.72208


In [ ]:
df.describe()

In [9]:
def get_unicode_codepoints(text):
    """获取日语文本中每个字符的Unicode码点（格式为U+XXXX）"""
    return "".join([f"{ord(char):04X}" for char in text])


japanese_word = "こんにちは"
get_unicode_codepoints(japanese_word)

'30533093306B3061306F'

In [19]:
df.dropna(subset=["headword", "type", "reading", "rank"], inplace=True)
# 2. 将次数映射到新列
value_counts = df["headword"].value_counts()
df["count"] = df["headword"].map(value_counts)
df = df[df["type"] != "記号"].copy()

df.sort_values(by="rank", ascending=False, inplace=True)
# df.index = df["text_column"].apply(get_unicode_codepoints)

In [26]:
df["hrt"] = df["headword"] + "-" + df["reading"] + "-" + df["type"]
hrt_counts = df["hrt"].value_counts()
df["hrt_count"] = df["hrt"].map(hrt_counts)
df["r"] = (
    df.groupby("hrt")["rank"]
    .rank(
        method="first", ascending=False
    )  # 并列时按出现顺序排名（类似 SQL 的 ROW_NUMBER）  # 升序排列（默认）
    .astype(int)
)


df

,headword,type,reading,freq,freq_per,rank,count,hrt,hrt_count,r
6995,の,助詞,ノ,"50,267,639",442.1104,0.99998,2,の-ノ-助詞,1,1
6673,に,助詞,ニ,"35,286,930",310.3531,0.99996,1,に-ニ-助詞,1,1
9550,を,助詞,ヲ,"33,707,106",296.4584,0.99995,1,を-ヲ-助詞,1,1
7122,は,助詞,ハ,"31,279,365",275.1061,0.99994,1,は-ハ-助詞,1,1
2665,が,助詞,ガ,"30,663,103",269.6860,0.99993,2,が-ガ-助詞,1,1
...,...,...,...,...,...,...,...,...,...,...
9567,をちゅうしんとして,助詞,ヲチュウシントシテ,1,0.0000,0.00007,1,をちゅうしんとして-ヲチュウシントシテ-助詞,1,1
9568,をちゅうしんに,助詞,ヲチュウシンニ,1,0.0000,0.00006,1,をちゅうしんに-ヲチュウシンニ-助詞,1,1
17476,バイオリニストする,動詞-自立,ヴァイオリニストスル,1,0.0000,0.00005,1,バイオリニストする-ヴァイオリニストスル-動詞-自立,1,1
17686,バリエーションする,動詞-自立,ヴァリエーションスル,1,0.0000,0.00004,2,バリエーションする-ヴァリエーションスル-動詞-自立,1,1


In [29]:
df[df['r']==1].to_csv(spoken_freq_nlb_csv_path, index=False, encoding="utf-8-sig")
print("ok")

ok


In [ ]:
def get_unicode_codepoints(text):
    """将文本中每个字符转换为4位十六进制Unicode码点字符串（无U+前缀）"""
    return "".join([f"{ord(char):04X}" for char in text])


# 创建示例 DataFrame
data = {
    "text_column": ["日本語", "こんにちは", "東京", "日本語"],  # 包含重复文本
    "value": [100, 200, 300, 400],
}
df = pd.DataFrame(data)

# 设置索引（处理重复值）
df.index = (
    df["text_column"].apply(get_unicode_codepoints)
    + "_"
    + df.groupby("text_column").cumcount().astype(str)
)

# 输出结果
print(df)

In [ ]:
with open("D:/05.japanproject/jp/freq/spoken_freq_nlb.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(f"检测到编码: {result['encoding']}")